## Raw Agent

Slightly improved based on https://learn.deeplearning.ai/courses/ai-agents-in-langgraph

In [18]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [19]:
from openai import OpenAI
from dotenv import load_dotenv
from functools import partial
import re
_ = load_dotenv()

In [20]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")


In [21]:
actions = [
    {
        "name": calculate,
        "usage": "e.g. calculate: 4 * 7 / 3",
        "desc": "Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary"
    },
    {
        "name": average_dog_weight,
        "usage": "e.g. average_dog_weight: Scottish Terrier",
        "desc": "Returns the average weight of a dog breed"
    }
]

In [22]:
prompt_template = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

{actions}
Here is a session template:

Question: [Users question]?
Thought: [Your thought to use an action]
Action: [selected action]: [action parameter]
PAUSE

You will be called again with the result of the action you selected::

Observation: [action result]

You then output:

Answer: [Your answer]
"""

In [23]:
class ReActAgent:
    def __init__(self, prompt_template, actions, openai_model = "gpt-4o-mini"):
        actions_in_prompt = ""
        for a in actions:
            actions_in_prompt += f"{a['name'].__name__}\n{a['usage']}\n{a['desc']}\n\n"

        self.system = prompt_template.format(actions = actions_in_prompt)
        self.messages = []
        self.messages.append({"role": "system", "content": self.system})
        self.chat = partial(OpenAI().chat.completions.create, model=openai_model, temperature=0)

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        completion = self.chat(messages=self.messages)
        out = completion.choices[0].message.content
        self.messages.append({"role": "assistant", "content": out})
        return out

In [24]:
agent = ReActAgent(prompt_template, actions)

**Checking the system prompt**

In [25]:
print(agent.system)


You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight
e.g. average_dog_weight: Scottish Terrier
Returns the average weight of a dog breed


Here is a session template:

Question: [Users question]?
Thought: [Your thought to use an action]
Action: [selected action]: [action parameter]
PAUSE

You will be called again with the result of the action you selected::

Observation: [action result]

You then output:

Answer: [Your answer]



**Testing Thought, Action, PAUSE**

In [26]:
out = agent("How much does a Toy Poodle weigh?")
print(out)


Question: How much does a Toy Poodle weigh?
Thought: I need to find the average weight of a Toy Poodle, so I will use the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


**Testing action call, Observation**

In [27]:
obs = f"Observation: {average_dog_weight("Toy Poodle")}"
obs

'Observation: a toy poodles average weight is 7 lbs'

In [28]:
out = agent(obs)
print(out)

Answer: A Toy Poodle weighs on average 7 lbs.


**List of messages**

In [29]:
agent.messages

[{'role': 'system',
  'content': '\nYou run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight\ne.g. average_dog_weight: Scottish Terrier\nReturns the average weight of a dog breed\n\n\nHere is a session template:\n\nQuestion: [Users question]?\nThought: [Your thought to use an action]\nAction: [selected action]: [action parameter]\nPAUSE\n\nYou will be called again with the result of the action you selected::\n\nObservation: [action result]\n\nYou then output:\n\nAnswer: [Your answer]\n'},
 {'role': 'user', 'content': 'How much does 

## Looping

**Parse action in the output**

In [30]:
action_re = re.compile(r"Action: (\w+): (.*)") 

In [31]:
ex_output = """
Question: How much does a Toy Poodle weigh?
Thought: I need to find the average weight of a Toy Poodle, so I will use the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE
"""
r = action_re.search(ex_output)

In [32]:
if(r):
    action, param = r.groups()

**Calling the selected action**

In [33]:
# Find the action with the name "average_dog_weight"
action = next((a for a in actions if a['name'].__name__ == action), None)

# Call the action if it exists
if action:
    result = action['name'](param)
    print(result)

a toy poodles average weight is 7 lbs


**Building all together**

In [34]:
def ask_agent(agent, question, max_turn=5):
    out = ""
    next_input_to_agent = question
    for i in range(max_turn):
        out = agent(next_input_to_agent)
        r = action_re.search(out)
        if not(r):
            # if there's no action to call then we are done
            break
        print("--- Agent thought process ------")
        print(out)
        print("--------------------------------")
        
        action, param = r.groups()
        action = next((a for a in actions if a['name'].__name__ == action), None)
        if not(action):
            raise Exception(f"Unknown action: {action}")
        next_input_to_agent = f"Observation: {action['name'](param)})"
    return out
    

In [35]:
agent = ReActAgent(prompt_template, actions)
answer = ask_agent(agent, "How much does a Toy Poodle weigh?")
print(answer)


--- Agent thought process ------
Question: How much does a Toy Poodle weigh?
Thought: I need to find the average weight of a Toy Poodle, so I will use the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE
--------------------------------
Answer: A Toy Poodle weighs on average 7 lbs.


In [36]:
agent = ReActAgent(prompt_template, actions)
answer = ask_agent(agent, "I have 2 dogs, a border collie and a scottish terrier.\
                 What is their combined weight")
print(answer)

--- Agent thought process ------
Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier to calculate their combined weight. I will first get the average weight of each breed. 
Action: average_dog_weight: Border Collie
PAUSE
--------------------------------
--- Agent thought process ------
Thought: Now that I have the average weight of the Border Collie, I need to find the average weight of the Scottish Terrier to calculate their combined weight. 
Action: average_dog_weight: Scottish Terrier
PAUSE
--------------------------------
--- Agent thought process ------
Thought: I now have the average weights of both dogs: the Border Collie is 37 lbs and the Scottish Terrier is 20 lbs. I will calculate their combined weight by adding these two weights together. 
Action: calculate: 37 + 20
PAUSE
--------------------------------
Answer: The combined weight of your Border Collie and Scottish Terrier is 57 lbs.
